In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
train = pd.read_csv("train_cleaned.csv")
X_test = pd.read_csv("test_cleaned.csv")

# initial version of test set for final result
test = pd.read_csv("test.csv")

In [3]:
# reclassify categorical variable
train['VENUE_KEY'] = train['VENUE_KEY'].astype('object', copy = False)
X_test['VENUE_KEY'] = X_test['VENUE_KEY'].astype('object', copy = False)

In [4]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [5]:
# preprocess
preprocess = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), make_column_selector(dtype_include = 'object')),
    remainder = StandardScaler())

X_train_scaled = preprocess.fit_transform(X_train)
X_test_scaled = preprocess.fit_transform(X_test)

In [6]:
# create pipeline for grid search of hyperparameters
pipe = Pipeline([('classifier', RandomForestClassifier())])

In [7]:
param_grid = [{'classifier': [RandomForestClassifier()],
             'classifier__n_estimators': [10,50,100],
             'classifier__min_samples_split': [2,4]},
             {'classifier': [GradientBoostingClassifier()],
             'classifier__learning_rate': [0.05,0.1,0.2],
             'classifier__max_depth': [2,3,4]},
             {'classifier': [KNeighborsClassifier()],
             'classifier__n_neighbors': [3,5,10]},
             {'classifier': [MLPClassifier()],
             'classifier__hidden_layer_sizes': [50,100]}]          

In [8]:
grid = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'neg_log_loss')

In [9]:
grid.fit(X_train_scaled, y_train);

C:\Users\Chris Russo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Chris Russo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Chris Russo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Chris Russo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

In [10]:
# use best performing classifier
grid.best_estimator_.get_params()['classifier']

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=2,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [11]:
# estimated probabilities of 5 outcomes
estimates = pd.DataFrame(grid.best_estimator_.predict_proba(X_test_scaled),
                        columns = ['double','homerun','other','single','triple'])

In [12]:
# drop 'other' and combine with initial table
results = pd.concat([test, estimates.loc[:,['single','double','triple','homerun']]], axis = 1)

In [13]:
# save
results.to_csv("test.csv", index = False)